# Анализ вакансий hh на дата-аналитика

In [266]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from  sklearn.ensemble import IsolationForest
import warnings

import requests
from pprint import pprint
from bs4 import BeautifulSoup
import re

warnings.filterwarnings('ignore')

## Создание датафрейма, парсинг информации

In [79]:
# подготовим пустой датафрейм с необходимыми полями
columns = 'id', 'name', 'insider_interview', 'response_letter_required', 'salary', 'address', 'allow_messages',\
     'experience', 'schedule', 'employment', 'department', 'contacts', 'description', 'key_skills', 'professional_roles', 'employer',\
     'published_at', 'created_at', 'apply_alternate_url', 'has_test', 'test', 'alternate_url',\
     'accept_temporary', 'languages'
df = pd.DataFrame(columns = columns)
df

,id,relations,name,insider_interview,response_letter_required,salary,type,address,allow_messages,experience,...,created_at,apply_alternate_url,has_test,test,alternate_url,working_days,working_time_intervals,working_time_modes,accept_temporary,languages


In [84]:
# общая страница с вакансиями
url = 'https://api.hh.ru/vacancies?search_field=name&search_field=company_name&search_field=description&text=data+scientist'
response = requests.get(url)
page_json = response.json()

# количество страниц
pages = page_json['pages']

18

In [91]:
#цикл по страницам с вакансиями
for page in range(pages):
    par = {'page':page}
    response = requests.get(url, params=par)
    page_json = response.json()

    # перебор всех вакансий на странице
    for item in page_json['items']:
        url_vacancy = item['url']   # переходим на страницу вакансии
        response = requests.get(url_vacancy)
        vacancy_json = response.json()
        dict_vacancy = {}

        for column in columns:
            dict_vacancy[column] = vacancy_json[column]

        df = df.append(dict_vacancy, ignore_index=True)

In [112]:
# сохраним в формат csv

df.to_csv('data/data_vacancy.csv', sep=';')

## Обработка данных

In [205]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        361 non-null    object
 1   relations                 361 non-null    object
 2   name                      361 non-null    object
 3   insider_interview         1 non-null      object
 4   response_letter_required  361 non-null    object
 5   salary                    63 non-null     object
 6   type                      361 non-null    object
 7   address                   147 non-null    object
 8   allow_messages            361 non-null    object
 9   experience                361 non-null    object
 10  schedule                  361 non-null    object
 11  employment                361 non-null    object
 12  department                123 non-null    object
 13  contacts                  0 non-null      object
 14  description               

#### 1. Обработка признака salary

In [198]:
def get_salary(text):

    if not text :
        return [0,0,'']

    dict_text = dict(text)
    salary_min = dict_text['from']
    salary_max = dict_text['to']
    salary_currency = dict_text['currency']
    print(salary_min)
    if salary_min is None:
        salary_min = salary_max

    if salary_max is None:
        salary_max = salary_min

    return [salary_min, salary_max, salary_currency]


In [199]:
df['salary_min'] = df['salary'].apply(lambda x: get_salary(x)[0])
df['salary_max'] = df['salary'].apply(lambda x: get_salary(x)[1])
df['salary_currency'] = df['salary'].apply(lambda x: get_salary(x)[2])

150000
5000
60000
None
40000
200000
150000
250000
400000
None
150000
100000
5000
None
40000
60000
200000
150000
250000
400000
None
200000
None
300000
None
100000
70000
350000
100000
50000
350000
3000
225000
3000
None
None
3000
120000
200000
150000
None
5200
200000
3500
3500
120000
3500
150000
40000
None
120000
400
7500
55000
45000
None
250000
20000
200000
None
3500
200000
200000
150000
5000
60000
None
40000
200000
150000
250000
400000
None
150000
100000
5000
None
40000
60000
200000
150000
250000
400000
None
200000
None
300000
None
100000
70000
350000
100000
50000
350000
3000
225000
3000
None
None
3000
120000
200000
150000
None
5200
200000
3500
3500
120000
3500
150000
40000
None
120000
400
7500
55000
45000
None
250000
20000
200000
None
3500
200000
200000
150000
5000
60000
None
40000
200000
150000
250000
400000
None
150000
100000
5000
None
40000
60000
200000
150000
250000
400000
None
200000
None
300000
None
100000
70000
350000
100000
50000
350000
3000
225000
3000
None
None
3000
120000
20

In [272]:
df.drop('salary', axis = 1, inplace=True)

#### 2. Обработка признака address

In [228]:
def get_address(text):

    if not text :
        return ['','','','','']

    dict_text = dict(text)
    city = dict_text['city']
    street = dict_text['street']
    lat = dict_text['lat']
    lng = dict_text['lng']
    if dict_text['metro'] and dict_text['metro']['station_name']:
        metro = dict_text['metro']['station_name']
    else:
        metro=''

    return [city, street, lat, lng, metro]

In [238]:
df['city'] = df['address'].apply(lambda x: get_address(x)[0])
df['street'] = df['address'].apply(lambda x: get_address(x)[1])
df['lat'] = df['address'].apply(lambda x: get_address(x)[2])
df['lng'] = df['address'].apply(lambda x: get_address(x)[3])
df['metro'] = df['address'].apply(lambda x: get_address(x)[4])

In [246]:
df = df.drop('address', axis = 1)

#### 3. Обработка признака experience

In [249]:
df['experience'] = df['experience'].apply(lambda x:dict(x)['name'] if x else '')

#### 3. Обработка признака schedule

In [251]:
df['schedule'] = df['schedule'].apply(lambda x:dict(x)['name'] if x else '')

#### 4. Обработка признака employment

In [252]:
df['employment'] = df['employment'].apply(lambda x:dict(x)['name'] if x else '')

#### 5. Обработка признака department

In [254]:
df['department'] = df['department'].apply(lambda x:dict(x)['name'] if x else '')

#### 6. Обработка признака professional_roles

In [262]:
df['professional_roles'] = df['professional_roles'].apply(lambda x:dict(x[0])['name'] if x else '')

#### 7. Обработка признака employer

In [263]:
df['employer_name'] = df['employer'].apply(lambda x:dict(x)['name'] if x else '')
df['employer_url'] = df['employer'].apply(lambda x:dict(x)['alternate_url'] if x else '')
df.drop('employer', axis=1, inplace=True)

#### 8. Обработка признака description

In [268]:
df['description'] = df['description'].apply(lambda x:(re.sub(r'\<[^>]*\>', '', x)) if x else '')

In [269]:
df['description']

0      Обязанности:  Работа в команде по созданию мод...
1      Наша компания «СтандартПроект» является лидеро...
2      FUNCORP, разработчик развлекательных UGC-прило...
3      Программист Python/Data Scientist (junior / mi...
4      Мы - бизнес-юнит &quot;Unity Fintech&quot;, ко...
                             ...                        
356    Обязанности:   Запуск автокредитования с нуля ...
357    The DataOps team is responsible for the buildi...
358    Hi there! We are Semrush, a global IT company ...
359    Команда AI Hub Group - это комьюнити талантлив...
360    Мы в Rubbles занимаемся созданием Data Science...
Name: description, Length: 361, dtype: object

In [275]:
df.drop(['relations', 'type'], axis=1, inplace=True)

In [276]:
df.head()

,id,name,insider_interview,response_letter_required,allow_messages,experience,schedule,employment,department,contacts,...,salary_min,salary_max,salary_currency,city,street,lat,lng,metro,employer_name,employer_url
0,69724262,Data scientist,None,False,True,От 3 до 6 лет,Удаленная работа,Полная занятость,,None,...,150000,200000,RUR,Москва,Походный проезд,55.838091,37.417167,,Астор,https://hh.ru/employer/606
1,70471850,Аналитик данных / Data scientist,None,False,True,От 1 года до 3 лет,Полный день,Полная занятость,,None,...,0,0,,Москва,улица Станиславского,55.745716,37.663539,,СтандартПроект,https://hh.ru/employer/2942396
2,67716744,Senior Data Scientist (Yepp),None,False,False,От 3 до 6 лет,Полный день,Полная занятость,,None,...,5000,8500,EUR,,,,,,FUNCORP,https://hh.ru/employer/1212374
3,69383038,Программист Python/Data Scientist (junior / mi...,None,False,True,Нет опыта,Полный день,Полная занятость,,None,...,60000,100000,RUR,,,,,,Апэрбот,https://hh.ru/employer/4602050
4,70373083,Data Scientist,None,False,True,От 3 до 6 лет,Удаленная работа,Полная занятость,,None,...,300000,300000,RUR,,,,,,Кадровое агентство Юнити,https://hh.ru/employer/985


In [273]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361 entries, 0 to 360
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   id                        361 non-null    object
 1   relations                 361 non-null    object
 2   name                      361 non-null    object
 3   insider_interview         1 non-null      object
 4   response_letter_required  361 non-null    object
 5   type                      361 non-null    object
 6   allow_messages            361 non-null    object
 7   experience                361 non-null    object
 8   schedule                  361 non-null    object
 9   employment                361 non-null    object
 10  department                361 non-null    object
 11  contacts                  0 non-null      object
 12  description               361 non-null    object
 13  key_skills                361 non-null    object
 14  professional_roles        

## EDA

In [283]:
df[(df['salary_min']!=0) & (df['salary_currency']=='RUR')][['name', 'alternate_url', 'salary_min', 'salary_max', 'city']].sort_values('salary_min')

,name,alternate_url,salary_min,salary_max,city
358,QA Engineer (Amber Team),https://hh.ru/vacancy/69423067,3500,3500,
346,French speaking recruitment intern,https://hh.ru/vacancy/70089172,20000,50000,Москва
5,Разработчик / программист / Data scientist,https://hh.ru/vacancy/69471079,40000,60000,Самара
25,Разработчик / программист / Data scientist,https://hh.ru/vacancy/69471079,40000,60000,Самара
293,Проджект-менеджер / Системный-аналитик в ИТ-пр...,https://hh.ru/vacancy/70341871,40000,40000,Москва
317,Специалист по системе 1С,https://hh.ru/vacancy/70498454,45000,45000,Красноярск
139,Data scientist,https://hh.ru/vacancy/69694041,50000,150000,
314,Менеджер по продажам курсов в сфере IT и Digit...,https://hh.ru/vacancy/67982126,55000,80000,Москва
3,Программист Python/Data Scientist (junior / mi...,https://hh.ru/vacancy/69383038,60000,100000,
26,Программист Python/Data Scientist (junior / mi...,https://hh.ru/vacancy/69383038,60000,100000,
